In [1]:
# Cell 0

network = "custom"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://18.144.108.38:9000",        "nid": 3}}

env = connections[network]

In [2]:
# Cell 1

from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from repeater import retry
from shutil import make_archive
import pickle as pkl
from datetime import datetime
from time import sleep
import json
import os

ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"
ORACLE = "cx61a36e5d10412e03c907a507d1e8c6c3856d9964"

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

In [3]:
# Cell 2

icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

In [4]:
# Cell 3

wallet = KeyWallet.load("keystores/keystore_test1.json", "test1_Account")
# Balanced test wallet
with open("keystores/balanced_test.pwd", "r") as f:
    key_data = f.read()
btest_wallet = KeyWallet.load("keystores/balanced_test.json", key_data)
print(icon_service.get_balance(wallet.get_address())/10**18)
print(icon_service.get_balance(btest_wallet.get_address())/10**18)

763283355.6477697
13874672.2959146


In [5]:
print(wallet.get_address())
print(icon_service.get_balance(wallet.get_address()) / 10**18)

hxe7af5fcfd8dfc67530a01a0e403882687528dfcb
763283355.6477697


In [6]:
print(btest_wallet.get_address())
print(icon_service.get_balance(btest_wallet.get_address()) / 10**18)

hx3f01840a599da07b0f620eeae7aa9c574169a4be
13874672.2959146


In [7]:
user1 = KeyWallet.load("keystores/user1.json","HelloWorld@1234")
# btest_wallet = KeyWallet.load("./balanced_test.json","HelloWorld@1234")

print(icon_service.get_balance(user1.get_address())/10**18)
print(user1.get_address())

# test2 = hx7a1824129a8fe803e45a3aae1c0e060399546187
private = "0a354424b20a7e3c55c43808d607bddfac85d033e63d7d093cb9f0a26c4ee022"
user2 = KeyWallet.load(bytes.fromhex(private))
print(icon_service.get_balance(user2.get_address())/10**18)
print(user2.get_address())

1046549.7415692
hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db
996194.951141
hx7a1824129a8fe803e45a3aae1c0e060399546187


In [8]:
# Cell 4
# Only necessary if running locally or on the private tbears server
# for the first time since reinitializing.

transaction = TransactionBuilder()\
    .from_(wallet.get_address())\
    .to(user2.get_address())\
    .value(10000 * ICX)\
    .step_limit(1000000) \
    .nid(NID) \
    .nonce(2) \
    .version(3) \
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

'0x61db4150bf5c8777219a21b722cc2d8bcabf48481dd51ba1e55cb80982fa1079'

In [8]:
# The following addresses are those deployed to the private tbears server.

contracts = {'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cxbbd36ca8d91aefbe1060c3be62fed4d1b848ca85'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cxd8e05c1280bc2c32bf53ff61f3bb2e2ecc7d6df5'},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': 'cx7c617e3fca4ba06b6ad203ce113245ae96a9d91e'},
 'reserve': {'zip': 'core_contracts/reserve.zip',
  'SCORE': 'cx70cd5c86f0182d5ac0bd224562d929cd968d9132'},
 'daofund': {'zip': 'core_contracts/daofund.zip',
  'SCORE': 'cxeb91bc377b0620356787d9c9eb68152eb0c62d8a'},
 'rewards': {'zip': 'core_contracts/rewards.zip',
  'SCORE': 'cxfd7511ece084744154fed19dc34732681ad078e6'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cx9a3161c778eee2d5758371d3548c5599f76704ec'},
 'governance': {'zip': 'core_contracts/governance.zip',
  'SCORE': 'cx238cd1a1e3a9702d6c9c6dc130719472164db376'},
 'oracle': {'zip': 'core_contracts/oracle.zip',
  'SCORE': 'cx7171e2f5653c1b9c000e24228276b8d24e84f10d'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cxcff8bf80ab213fa9bbb350636a4d68f5cb4fd9c1'},
 'bnUSD': {'zip': 'token_contracts/bnUSD.zip',
  'SCORE': 'cx4c1beaa71b9377100c810c46059ddf5f3da37602'},
 'baln': {'zip': 'token_contracts/baln.zip',
  'SCORE': 'cx3825a86d52c5baf188ff29aa6a7fc2467e285885'},
 'bwt': {'zip': 'token_contracts/bwt.zip',
  'SCORE': 'cx140b49ea041457ebc4cd5e199f5723916bb50021'}}

In [9]:
contracts = {'loans': {'zip': 'core_contracts/loans.zip', 'SCORE': 'cx764463174b4f7d44988968f93c08081881fcfc85'}, 'staking': {'zip': 'core_contracts/staking.zip', 'SCORE': 'cx686423d2293831a6cb2441959112a1a06b7462ba'}, 'dividends': {'zip': 'core_contracts/dividends.zip', 'SCORE': 'cxa6f208b9b7fecd68e7cf66675ba058ef68c00ef6'}, 'reserve': {'zip': 'core_contracts/reserve.zip', 'SCORE': 'cxedf87c1cd6811891016606c403418cf967340a51'}, 'daofund': {'zip': 'core_contracts/daofund.zip', 'SCORE': 'cxadedc487277793fe7cd73ac4cdfdbc2fee02e0e1'}, 'rewards': {'zip': 'core_contracts/rewards.zip', 'SCORE': 'cxbbf173efedf4338d0562edf67f3f563cd6f09b18'}, 'dex': {'zip': 'core_contracts/dex.zip', 'SCORE': 'cx2d8cd99b2ed00a8ec519a00a9d42dd314e72a04b'}, 'governance': {'zip': 'core_contracts/governance.zip', 'SCORE': 'cxbdfe800a34762f3e65bdef1be4f3058b893d6fe5'}, 'oracle': {'zip': 'core_contracts/oracle.zip', 'SCORE': 'cx7171e2f5653c1b9c000e24228276b8d24e84f10d'}, 'sicx': {'zip': 'token_contracts/sicx.zip', 'SCORE': 'cxf99f4ca65fda1cdfa461f473d60f729341524022'}, 'bnUSD': {'zip': 'token_contracts/bnUSD.zip', 'SCORE': 'cx0f493f0d561d14576f3705ff422c32fe305b2e6a'}, 'baln': {'zip': 'token_contracts/baln.zip', 'SCORE': 'cxdf5d7067e425b3e1a2c96659b3fd91bd1b5dd0d9'}, 'bwt': {'zip': 'token_contracts/bwt.zip', 'SCORE': 'cxf2c45e05c329bd9c0dd7780b8f14a15a1eaa78ff'}}

# Deploy All Scores

In [10]:
# Cell 6
# Define deploy and send_tx functions

def compress():
    """
    Compress all SCORE folders in the core_contracts and toekn_contracts folders
    """
    deploy = list(contracts.keys())[:]
    for directory in {"core_contracts", "token_contracts"}:
        with os.scandir(directory) as it:
            for file in it:
                archive_name = directory + "/" + file.name
                if file.is_dir() and file.name in deploy:
                    make_archive(archive_name, "zip", directory, file.name)
                    contracts[file.name]['zip'] = archive_name + '.zip'

def deploy_SCORE(contract, params, wallet, update) -> str:
    """
    contract is of form {'zip': 'core_contracts/governance.zip', 'SCORE': 'cx1d81f93b3b8d8d2a6455681c46128868782ddd09'}
    params is a dicts
    wallet is a wallet file
    update is boolian
    """
    print(f'{contract["zip"]}')
    if update:
        dest = contract['SCORE']
    else:
        dest = GOVERNANCE_ADDRESS
    zip_file = contract['zip']
    step_limit = 4000100000
    deploy_transaction = DeployTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(dest)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content(zip_file))\
        .params(params)\
        .build()

    signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    print(f'Status: {res["status"]}')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        print(f'Failure: {res["failure"]}')
    print('')
    return res.get('scoreAddress', '')

def send_tx(dest, value, method, params, wallet):
    """
    dest is the name of the destination contract.
    """
    print('------------------------------------------------------------------------------------------------------------------')
    print(f'Calling {method}, with parameters {params} on the {dest} contract.')
    print('------------------------------------------------------------------------------------------------------------------')
    transaction = CallTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[dest]['SCORE'])\
        .value(value)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(method)\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    print(f'************************************************** Status: {res["status"]} **************************************************')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        print(f'Failure: {res["failure"]}')
    return res

def deploy_all(wallet):
    """
    Compress, Deploy and Configure all SCOREs
    """
    compress()

    deploy = list(contracts.keys())[:]
    deploy.remove('oracle')
    deploy.remove('staking')
    deploy.remove('sicx')
    deploy.remove('governance')

    governance = deploy_SCORE(contracts['governance'], {}, wallet, 0)
    contracts['governance']['SCORE'] = governance
    for score in deploy:
        contracts[score]['SCORE'] = deploy_SCORE(contracts[score], {'_governance': governance}, wallet, 0)
    contracts['staking']['SCORE'] = deploy_SCORE(contracts['staking'], {}, wallet, 0)
    contracts['sicx']['SCORE'] = deploy_SCORE(contracts['sicx'], {'_admin': contracts['staking']['SCORE']}, wallet, 0)

    config = list(contracts.keys())[:]
    config.remove('governance')
    addresses = {contract: contracts[contract]['SCORE'] for contract in config}

    txns = [{'contract': 'staking', 'value': 0, 'method': 'setSicxAddress', 'params': {'_address': contracts['sicx']['SCORE']}},
            {'contract': 'governance', 'value': 0, 'method': 'setAddresses', 'params': {'_addresses': addresses}},
            {'contract': 'governance', 'value': 0, 'method': 'launchBalanced', 'params': {}}]

    for tx in txns:
        res = send_tx(tx["contract"], tx["value"], tx["method"], tx["params"], wallet)
        results[f'{tx["contract"]}|{tx["method"]}|{tx["params"]}'] = res

def get_scores_json(contracts):
    """
    Prints out dictionary of SCORE addresses for use in testing UI.
    """
    scores = {}
    for score in contracts:
        scores[score] = contracts[score]['SCORE']
    return json.dumps(scores)

def call_tx(dest: str, method: str, params: dict = {}):
    """
    dest is the name of the destination contract.
    """
    print('------------------------------------------------------------------------------------------------------------------')
    print(f'Reading {method}, with parameters {params} on the {dest} contract.')
    print('------------------------------------------------------------------------------------------------------------------')
    call = CallBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[dest]['SCORE'])\
        .method(method)\
        .params(params)\
        .build()
    print(icon_service.call(call))
    return icon_service.call(call)


In [10]:
# Cell 7
# Deploy and configure Balanced. Print results if anything goes wrong.

results = {}
deploy_all(btest_wallet)
print('------------------------------------------------------------------------------------------------------------------')
print(contracts)
print(get_scores_json(contracts))

core_contracts/governance.zip


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
Status: 1

core_contracts/loans.zip


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
Status: 1

core_contracts/dividends.zip


...

Retrying in 1 seconds...
Status: 1

core_contracts/daofund.zip


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
Status: 1

core_contracts/rewards.zip
Status: 1

core_contracts/dex.zip
Status: 1

token_contracts/bnUSD.zip
Status: 1

token_contracts/baln.zip
Status: 1

token_contracts/bwt.zip
Status: 1

core_contracts/staking.zip
Status: 1

token_contracts/sicx.zip
Status: 1

------------------------------------------------------------------------------------------------------------------
Calling setSicxAddress, with parameters {'_address': 'cxf99f4ca65fda1cdfa461f473d60f729341524022'} on the staking contract.
------------------------------------------------------------------------------------------------------------------
************************************************** Status: 1 **************************************************
------------------------------------------------------------------------------------------------------------------
Calling setAddresses, with parameters {'_addresses': {'loans': 'cx764463174b4f7d44988968f93c08081881fcfc85', 'staking': 'cx686423

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cx764463174b4f7d44988968f93c08081881fcfc85', 'indexed': ['AssetAdded(Address,str,bool)', 'cxf99f4ca65fda1cdfa461f473d60f729341524022', 'sICX', '0x1'], 'data': []} 

{'scoreAddress': 'cx764463174b4f7d44988968f93c08081881fcfc85', 'indexed': ['AssetAdded(Address,str,bool)', 'cx0f493f0d561d14576f3705ff422c32fe305b2e6a', 'bnUSD', '0x0'], 'data': []} 

{'scoreAddress': 'cx764463174b4f7d44988968f93c08081881fcfc85', 'indexed': ['AssetAdded(Address,str,bool)', 'cxdf5d7067e425b3e1a2c96659b3fd91bd1b5dd0d9', 'BALN', '0x1'], 'data': []} 

{'scoreAddress': 'cx764463174b4f7d44988968f93c08081881fcfc85', 'indexed': ['ContractActive(str,str)', 'Loans'], 'data': ['Active']} 

------------------------------------------------------------------------------------------------------------------
{'loans': {'zip': 'core_contracts/loans.zip', 'SCORE': 'cx764463

Update a score

In [14]:
# Cell 8
# Deploy or Update a single SCORE

compress()
update = 1
contract = contracts['loans']
params = {}
# params = {'_governance': contracts['governance']['SCORE']}
deploy_SCORE(contract, params, btest_wallet, update)

core_contracts/loans.zip


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
Status: 1



'cx764463174b4f7d44988968f93c08081881fcfc85'

Test getDebts method

In [12]:
# 1. Deposit collateral and mint loan from different accounts
# 2. call getDebts method on loans

send_tx('loans', 2500 * ICX , 'addCollateral', {'_asset': 'bnUSD', '_amount': 500 * ICX}, user1)
send_tx('loans', 2500 * ICX , 'addCollateral', {'_asset': 'bnUSD', '_amount': 1000 * ICX}, user2)
call_tx('loans', 'getAccountPositions', {'_owner': user1.get_address()})
call_tx('loans', 'getAccountPositions', {'_owner': user2.get_address()})



------------------------------------------------------------------------------------------------------------------
Calling addCollateral, with parameters {'_asset': 'bnUSD', '_amount': 500000000000000000000} on the loans contract.
------------------------------------------------------------------------------------------------------------------


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cxb2729d8ec932b0701e1a7d731ee1bf0f86e8a00d', 'indexed': ['ICXTransfer(Address,Address,int)', 'cxb2729d8ec932b0701e1a7d731ee1bf0f86e8a00d', 'cxbe35638c3cf1773ca533866c47964b5ff61c1065', '0x878678326eac900000'], 'data': []} 

{'scoreAddress': 'cxa1d8682563a3b7087fcd174d0448054cc0279347', 'indexed': ['Mint(Address,int,bytes)', 'cxb2729d8ec932b0701e1a7d731ee1bf0f86e8a00d'], 'data': ['0x878678326eac900000', '0x7b226d6574686f64223a225f6465706f7369745f616e645f626f72726f77222c22706172616d73223a7b225f73656e646572223a22687865363264313166613139613065383537356164393266303662633866643432656462666532376462222c225f6173736574223a22626e555344222c225f616d6f756e74223a3530303030303030303030303030303030303030307d7d']} 

{'scoreAddress': 'cxa1d8682563a3b7087fcd174d0448054cc0279347', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hx0000000000000000000

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cxb2729d8ec932b0701e1a7d731ee1bf0f86e8a00d', 'indexed': ['ICXTransfer(Address,Address,int)', 'cxb2729d8ec932b0701e1a7d731ee1bf0f86e8a00d', 'cxbe35638c3cf1773ca533866c47964b5ff61c1065', '0x878678326eac900000'], 'data': []} 

{'scoreAddress': 'cxa1d8682563a3b7087fcd174d0448054cc0279347', 'indexed': ['Mint(Address,int,bytes)', 'cxb2729d8ec932b0701e1a7d731ee1bf0f86e8a00d'], 'data': ['0x878678326eac900000', '0x7b226d6574686f64223a225f6465706f7369745f616e645f626f72726f77222c22706172616d73223a7b225f73656e646572223a22687837613138323431323961386665383033653435613361616531633065303630333939353436313837222c225f6173736574223a22626e555344222c225f616d6f756e74223a313030303030303030303030303030303030303030307d7d']} 

{'scoreAddress': 'cxa1d8682563a3b7087fcd174d0448054cc0279347', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hx00000000000000000

{'pos_id': '0x2',
 'created': '0x5becde15b7b02',
 'address': 'hx7a1824129a8fe803e45a3aae1c0e060399546187',
 'snap_id': '0x1',
 'snaps_length': '0x1',
 'last_snap': '0x1',
 'first day': '0x1',
 'assets': {'sICX': '0x878678326eac900000', 'bnUSD': '0x36c090d0ca68880000'},
 'replay_index': '0x0',
 'events_behind': '0x0',
 'total_debt': '0x0',
 'ratio': '0x397284386e139261',
 'standing': 'Not Mining'}

In [21]:
# call getDebts method
call_tx('loans', 'getDebts', {'_address_list':[user1.get_address(), user2.get_address()], '_day': 1})


------------------------------------------------------------------------------------------------------------------
Reading getDebts, with parameters {'_address_list': ['hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db', 'hx7a1824129a8fe803e45a3aae1c0e060399546187'], '_day': -1} on the loans contract.
------------------------------------------------------------------------------------------------------------------
{'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db': '0x105ea492156ae494e3', 'hx7a1824129a8fe803e45a3aae1c0e060399546187': '0x20bd49242ad5c929c6'}


{'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db': '0x105ea492156ae494e3',
 'hx7a1824129a8fe803e45a3aae1c0e060399546187': '0x20bd49242ad5c929c6'}

# Test Originate Loan 

In [22]:
#Try originating loans with an account that doesn’t have a position in Balanced 
call_tx('loans', 'getAccountPositions', {'_owner': btest_wallet.get_address()})
send_tx('loans', 0, 'originateLoan', {'_asset':'bnUSD', '_amount':50 *ICX}, btest_wallet)


------------------------------------------------------------------------------------------------------------------
Reading getAccountPositions, with parameters {'_owner': 'hx3f01840a599da07b0f620eeae7aa9c574169a4be'} on the loans contract.
------------------------------------------------------------------------------------------------------------------
{'message': 'That address has no outstanding loans or deposited collateral.'}
------------------------------------------------------------------------------------------------------------------
Calling originateLoan, with parameters {'_asset': 'bnUSD', '_amount': 50000000000000000000} on the loans contract.
------------------------------------------------------------------------------------------------------------------


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 0 **************************************************
Failure: {'code': 32, 'message': 'This address does not have a position on Balanced. Collateral must be deposited before originating a loan.'}


{'txHash': '0xad1bef79cecc619cf2086db3079f32dc4a42129c1587d2e98f5b3622343fa180',
 'blockHeight': 1870684,
 'blockHash': '0x656800d205f58d72a23e1bcde59a33ebbcda8ad5d7af823c5ac6710c3364a217',
 'txIndex': 1,
 'to': 'cxb2729d8ec932b0701e1a7d731ee1bf0f86e8a00d',
 'stepUsed': 142200,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 142200,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [23]:
# Try depositing 10 ICX and call originate loan of 50 bnUSD
call_tx('loans', 'getAccountPositions', {'_owner': btest_wallet.get_address()})
send_tx('loans', 10*ICX, 'addCollateral', {'_asset': '', '_amount': 0}, btest_wallet)
send_tx('loans', 0, 'originateLoan', {'_asset':'bnUSD', '_amount':50 *ICX}, btest_wallet)


------------------------------------------------------------------------------------------------------------------
Reading getAccountPositions, with parameters {'_owner': 'hx3f01840a599da07b0f620eeae7aa9c574169a4be'} on the loans contract.
------------------------------------------------------------------------------------------------------------------
{'message': 'That address has no outstanding loans or deposited collateral.'}
------------------------------------------------------------------------------------------------------------------
Calling addCollateral, with parameters {'_asset': '', '_amount': 0} on the loans contract.
------------------------------------------------------------------------------------------------------------------


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cxb2729d8ec932b0701e1a7d731ee1bf0f86e8a00d', 'indexed': ['ICXTransfer(Address,Address,int)', 'cxb2729d8ec932b0701e1a7d731ee1bf0f86e8a00d', 'cxbe35638c3cf1773ca533866c47964b5ff61c1065', '0x8ac7230489e80000'], 'data': []} 

{'scoreAddress': 'cxa1d8682563a3b7087fcd174d0448054cc0279347', 'indexed': ['Mint(Address,int,bytes)', 'cxb2729d8ec932b0701e1a7d731ee1bf0f86e8a00d'], 'data': ['0x8ac7230489e80000', '0x7b226d6574686f64223a225f6465706f7369745f616e645f626f72726f77222c22706172616d73223a7b225f73656e646572223a22687833663031383430613539396461303762306636323065656165376161396335373431363961346265222c225f6173736574223a22222c225f616d6f756e74223a307d7d']} 

{'scoreAddress': 'cxa1d8682563a3b7087fcd174d0448054cc0279347', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hx0000000000000000000000000000000000000000', 'cxb2729d8ec932b0701e1a7d731ee

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 0 **************************************************
Failure: {'code': 32, 'message': '10.0 collateral is insufficient to originate a loan of 50.0 bnUSD when max_debt_value = 2.5 and new_debt_value = 30.196764153578393, plus a fee of 0.5 bnUSD, and an existing loan value of 0.0.'}


{'txHash': '0xd48343c95fe95a87e605eea73d4f154124933e62021c68bf562c50c9aa01745c',
 'blockHeight': 1871297,
 'blockHash': '0x88dca209cba0ec09b12552ae477782efc4df963524362a48817edee08bb87a58',
 'txIndex': 1,
 'to': 'cxb2729d8ec932b0701e1a7d731ee1bf0f86e8a00d',
 'stepUsed': 683840,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 683840,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [25]:
# Try depositing enough ICX to get a loan of 50 bnUSD and call originate loan of 50 bnUSD 
call_tx('loans', 'getAccountPositions', {'_owner': btest_wallet.get_address()})
send_tx('loans', 250*ICX, 'addCollateral', {'_asset': '', '_amount': 0}, btest_wallet)
send_tx('loans', 0, 'originateLoan', {'_asset':'bnUSD', '_amount':50 *ICX}, btest_wallet)
call_tx('loans', 'getAccountPositions', {'_owner': btest_wallet.get_address()})


------------------------------------------------------------------------------------------------------------------
Reading getAccountPositions, with parameters {'_owner': 'hx3f01840a599da07b0f620eeae7aa9c574169a4be'} on the loans contract.
------------------------------------------------------------------------------------------------------------------
{'pos_id': '0x3', 'created': '0x5becf224944ab', 'address': 'hx3f01840a599da07b0f620eeae7aa9c574169a4be', 'snap_id': '0x3', 'snaps_length': '0x2', 'last_snap': '0x3', 'first day': '0x2', 'assets': {'sICX': '0x2b5e3af16b1880000'}, 'replay_index': '0x0', 'events_behind': '0x0', 'total_debt': '0x0', 'ratio': '0x0', 'standing': 'No Debt'}
------------------------------------------------------------------------------------------------------------------
Calling addCollateral, with parameters {'_asset': '', '_amount': 0} on the loans contract.
-------------------------------------------------------------------------------------------------------

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cxb2729d8ec932b0701e1a7d731ee1bf0f86e8a00d', 'indexed': ['OriginateLoan(Address,str,int,str)', 'hx3f01840a599da07b0f620eeae7aa9c574169a4be', 'bnUSD', '0x2b5e3af16b1880000'], 'data': ['Loan of 50.0 bnUSD from Balanced.']} 

{'scoreAddress': 'cx78f877bbe7e86d725aaed5e9a0f30f7ae3598abf', 'indexed': ['Mint(Address,int,bytes)', 'hx3f01840a599da07b0f620eeae7aa9c574169a4be'], 'data': ['0x2b5e3af16b1880000', '0x4e6f6e65']} 

{'scoreAddress': 'cx78f877bbe7e86d725aaed5e9a0f30f7ae3598abf', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hx0000000000000000000000000000000000000000', 'hx3f01840a599da07b0f620eeae7aa9c574169a4be', '0x2b5e3af16b1880000'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cx78f877bbe7e86d725aaed5e9a0f30f7ae3598abf', 'indexed': ['Mint(Address,int,bytes)', 'cx50e9677ad4f98190bba47281aa61a313e8e69fa8'], 'data': ['0x6f05b59

{'pos_id': '0x3',
 'created': '0x5becf224944ab',
 'address': 'hx3f01840a599da07b0f620eeae7aa9c574169a4be',
 'snap_id': '0x3',
 'snaps_length': '0x2',
 'last_snap': '0x3',
 'first day': '0x2',
 'assets': {'sICX': '0x1043561a8829300000', 'bnUSD': '0x2bcd40a70853a0000'},
 'replay_index': '0x0',
 'events_behind': '0x0',
 'total_debt': '0x0',
 'ratio': '0x89dfa3baa1c89284',
 'standing': 'Mining'}

In [26]:
call_tx('loans', 'getAccountPositions', {'_owner': wallet.get_address()})
call_tx('loans', 'getAccountPositions', {'_owner': btest_wallet.get_address()})


------------------------------------------------------------------------------------------------------------------
Reading getAccountPositions, with parameters {'_owner': 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb'} on the loans contract.
------------------------------------------------------------------------------------------------------------------
{'message': 'That address has no outstanding loans or deposited collateral.'}
------------------------------------------------------------------------------------------------------------------
Reading getAccountPositions, with parameters {'_owner': 'hx3f01840a599da07b0f620eeae7aa9c574169a4be'} on the loans contract.
------------------------------------------------------------------------------------------------------------------
{'pos_id': '0x3', 'created': '0x5becf224944ab', 'address': 'hx3f01840a599da07b0f620eeae7aa9c574169a4be', 'snap_id': '0x3', 'snaps_length': '0x2', 'last_snap': '0x3', 'first day': '0x2', 'assets': {'sICX': '0x1043

{'pos_id': '0x3',
 'created': '0x5becf224944ab',
 'address': 'hx3f01840a599da07b0f620eeae7aa9c574169a4be',
 'snap_id': '0x3',
 'snaps_length': '0x2',
 'last_snap': '0x3',
 'first day': '0x2',
 'assets': {'sICX': '0x1043561a8829300000', 'bnUSD': '0x2bcd40a70853a0000'},
 'replay_index': '0x0',
 'events_behind': '0x0',
 'total_debt': '0x0',
 'ratio': '0x89dfa3baa1c89284',
 'standing': 'Mining'}

Trying to mint loan to account wallet from btest_wallet

In [28]:
send_tx('loans', 100*ICX, 'addCollateral', {'_asset': '', '_amount': 0}, wallet)
send_tx('loans', 0, 'originateLoan', {'_asset':'bnUSD', '_amount':50 *ICX, '_from': wallet.get_address()}, btest_wallet)
call_tx('loans', 'getAccountPositions', {'_owner': btest_wallet.get_address()})
call_tx('loans', 'getAccountPositions', {'_owner': wallet.get_address()})



------------------------------------------------------------------------------------------------------------------
Calling addCollateral, with parameters {'_asset': '', '_amount': 0} on the loans contract.
------------------------------------------------------------------------------------------------------------------


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cxb2729d8ec932b0701e1a7d731ee1bf0f86e8a00d', 'indexed': ['ICXTransfer(Address,Address,int)', 'cxb2729d8ec932b0701e1a7d731ee1bf0f86e8a00d', 'cxbe35638c3cf1773ca533866c47964b5ff61c1065', '0x56bc75e2d63100000'], 'data': []} 

{'scoreAddress': 'cxa1d8682563a3b7087fcd174d0448054cc0279347', 'indexed': ['Mint(Address,int,bytes)', 'cxb2729d8ec932b0701e1a7d731ee1bf0f86e8a00d'], 'data': ['0x56bc75e2d63100000', '0x7b226d6574686f64223a225f6465706f7369745f616e645f626f72726f77222c22706172616d73223a7b225f73656e646572223a22687865376166356663666438646663363735333061303161306534303338383236383735323864666362222c225f6173736574223a22222c225f616d6f756e74223a307d7d']} 

{'scoreAddress': 'cxa1d8682563a3b7087fcd174d0448054cc0279347', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hx0000000000000000000000000000000000000000', 'cxb2729d8ec932b0701e1a7d731

{'pos_id': '0x4',
 'created': '0x5bed02919041f',
 'address': 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb',
 'snap_id': '0x3',
 'snaps_length': '0x1',
 'last_snap': '0x3',
 'first day': '0x3',
 'assets': {'sICX': '0xad78ebc5ac6200000', 'bnUSD': '0x2bcd40a70853a0000'},
 'replay_index': '0x0',
 'events_behind': '0x0',
 'total_debt': '0x0',
 'ratio': '0x5bea6d271685b703',
 'standing': 'Mining'}

Try taking loan from user1 who has deposited 200 ICX and havent taken loan, calling from btest_wallet

In [29]:
# 1. Deposit only collateral to user1 wallet of 200 ICX
call_tx('loans', 'getAccountPositions', {'_owner': user1.get_address()})
send_tx('loans', 200*ICX, 'addCollateral', {'_asset': '', '_amount': 0}, user1)
call_tx('loans', 'getAccountPositions', {'_owner': user1.get_address()})
send_tx('loans', 0, 'originateLoan', {'_asset':'bnUSD', '_amount':50 *ICX, '_from': user1.get_address()}, btest_wallet)
call_tx('loans', 'getAccountPositions', {'_owner': user1.get_address()})



------------------------------------------------------------------------------------------------------------------
Reading getAccountPositions, with parameters {'_owner': 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db'} on the loans contract.
------------------------------------------------------------------------------------------------------------------
{'pos_id': '0x1', 'created': '0x5becde11e6347', 'address': 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db', 'snap_id': '0x1', 'snaps_length': '0x1', 'last_snap': '0x1', 'first day': '0x1', 'assets': {'sICX': '0x878678326eac900000', 'bnUSD': '0x1b6048686534440000'}, 'replay_index': '0x0', 'events_behind': '0x0', 'total_debt': '0x105ea492156ae494e3', 'ratio': '0x0', 'standing': 'Indeterminate'}
------------------------------------------------------------------------------------------------------------------
Calling addCollateral, with parameters {'_asset': '', '_amount': 0} on the loans contract.
----------------------------------------------

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cxb2729d8ec932b0701e1a7d731ee1bf0f86e8a00d', 'indexed': ['OriginateLoan(Address,str,int,str)', 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db', 'bnUSD', '0x2b5e3af16b1880000'], 'data': ['Loan of 50.0 bnUSD from Balanced.']} 

{'scoreAddress': 'cx78f877bbe7e86d725aaed5e9a0f30f7ae3598abf', 'indexed': ['Mint(Address,int,bytes)', 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db'], 'data': ['0x2b5e3af16b1880000', '0x4e6f6e65']} 

{'scoreAddress': 'cx78f877bbe7e86d725aaed5e9a0f30f7ae3598abf', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hx0000000000000000000000000000000000000000', 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db', '0x2b5e3af16b1880000'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cx78f877bbe7e86d725aaed5e9a0f30f7ae3598abf', 'indexed': ['Mint(Address,int,bytes)', 'cx50e9677ad4f98190bba47281aa61a313e8e69fa8'], 'data': ['0x6f05b59

{'pos_id': '0x1',
 'created': '0x5becde11e6347',
 'address': 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db',
 'snap_id': '0x4',
 'snaps_length': '0x2',
 'last_snap': '0x4',
 'first day': '0x1',
 'assets': {'sICX': '0x925e06eec972b00000', 'bnUSD': '0x1e1d1c72d5b97e0000'},
 'replay_index': '0x0',
 'events_behind': '0x0',
 'total_debt': '0x0',
 'ratio': '0x70ce402455d2a66c',
 'standing': 'Mining'}

# Retiring  an asset

In [11]:
# 1.no bad debt for asset, retired from account user2 without a position on Balanced.
call_tx('loans', 'getAccountPositions', {'_owner': user1.get_address()})
call_tx('loans', 'getAccountPositions', {'_owner': user2.get_address()})


------------------------------------------------------------------------------------------------------------------
Reading getAccountPositions, with parameters {'_owner': 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db'} on the loans contract.
------------------------------------------------------------------------------------------------------------------
{'pos_id': '0x1', 'created': '0x5bf86488e38e9', 'address': 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db', 'snap_id': '0x4', 'snaps_length': '0x3', 'last_snap': '0x4', 'first day': '0x2', 'assets': {'sICX': '0xa2a15d09519be00000', 'bnUSD': '0x2c04c381d6f1300000'}, 'total_debt': '0x1a5238c66456b60124', 'collateral': '0xa2a15d09519be00000', 'ratio': '0x55bf1b1b7aaf7fb1', 'standing': 'Mining'}
------------------------------------------------------------------------------------------------------------------
Reading getAccountPositions, with parameters {'_owner': 'hx7a1824129a8fe803e45a3aae1c0e060399546187'} on the loans contract.
-------------

{'message': 'That address has no outstanding loans or deposited collateral.'}

In [25]:
call_tx('loans', 'getMaxRetireAmount', {'_symbol': 'bnUSD'})


------------------------------------------------------------------------------------------------------------------
Reading getMaxRetireAmount, with parameters {'_symbol': 'bnUSD'} on the loans contract.
------------------------------------------------------------------------------------------------------------------
0xb4b2ff235b2ae000


'0xb4b2ff235b2ae000'

In [24]:
send_tx('loans', 1000*ICX, 'addCollateral', {'_asset': 'bnUSD', '_amount': 500 * ICX}, user1)


------------------------------------------------------------------------------------------------------------------
Calling addCollateral, with parameters {'_asset': 'bnUSD', '_amount': 500000000000000000000} on the loans contract.
------------------------------------------------------------------------------------------------------------------


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cx764463174b4f7d44988968f93c08081881fcfc85', 'indexed': ['ICXTransfer(Address,Address,int)', 'cx764463174b4f7d44988968f93c08081881fcfc85', 'cx686423d2293831a6cb2441959112a1a06b7462ba', '0x3635c9adc5dea00000'], 'data': []} 

{'scoreAddress': 'cx0000000000000000000000000000000000000000', 'indexed': ['IScoreClaimedV2(Address,int,int)', 'cx686423d2293831a6cb2441959112a1a06b7462ba'], 'data': ['0x1c8fde30ee35fd13da8', '0x74fd693b1b7ce199']} 

{'scoreAddress': 'cxf99f4ca65fda1cdfa461f473d60f729341524022', 'indexed': ['Mint(Address,int,bytes)', 'cx764463174b4f7d44988968f93c08081881fcfc85'], 'data': ['0x3635c9adc5dea00000', '0x7b226d6574686f64223a225f6465706f7369745f616e645f626f72726f77222c22706172616d73223a7b225f73656e646572223a22687865363264313166613139613065383537356164393266303662633866643432656462666532376462222c225f6173736574223a22626e5

{'txHash': '0xa2ed5c6adf4570fccdb263a44d0e715039b8e15589af3d2202c2c67af4411a44',
 'blockHeight': 2389608,
 'blockHash': '0x10be199ff5c171e45a466b50eec114d1b0dd8fc640434183390a27e78c53917e',
 'txIndex': 1,
 'to': 'cx764463174b4f7d44988968f93c08081881fcfc85',
 'stepUsed': 1578860,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1578860,
 'eventLogs': [{'scoreAddress': 'cx764463174b4f7d44988968f93c08081881fcfc85',
   'indexed': ['ICXTransfer(Address,Address,int)',
    'cx764463174b4f7d44988968f93c08081881fcfc85',
    'cx686423d2293831a6cb2441959112a1a06b7462ba',
    '0x3635c9adc5dea00000'],
   'data': []},
  {'scoreAddress': 'cx0000000000000000000000000000000000000000',
   'indexed': ['IScoreClaimedV2(Address,int,int)',
    'cx686423d2293831a6cb2441959112a1a06b7462ba'],
   'data': ['0x1c8fde30ee35fd13da8', '0x74fd693b1b7ce199']},
  {'scoreAddress': 'cxf99f4ca65fda1cdfa461f473d60f729341524022',
   'indexed': ['Mint(Address,int,bytes)',
    'cx764463174b4f7d44988968f93c08081881fcfc85'],
 

In [26]:
# redeem 50 ICD from user2 that do not have position on Balanced. This will use up all
# of the liquidation pool and require some replay events to be recorded.


params = {'_to': user2.get_address(), '_value': 10 * ICX}
transaction = CallTransactionBuilder()\
    .from_(user1.get_address())\
    .to(contracts['bnUSD']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, user1)
tx_hash = icon_service.send_transaction(signed_transaction)
sleep(2)
params = {"method": "_retire_asset", "params": {}}
data = json.dumps(params).encode("utf-8")
params = {'_to': contracts['loans']['SCORE'], '_value': 10 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(user2.get_address())\
    .to(contracts['bnUSD']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, user2)
tx_hash = icon_service.send_transaction(signed_transaction)
sleep(1)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Status: 1
{'scoreAddress': 'cx0f493f0d561d14576f3705ff422c32fe305b2e6a', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hx7a1824129a8fe803e45a3aae1c0e060399546187', 'cx764463174b4f7d44988968f93c08081881fcfc85', '0x8ac7230489e80000'], 'data': ['0x7b226d6574686f64223a20225f7265746972655f6173736574222c2022706172616d73223a207b7d7d']} 

{'scoreAddress': 'cx0f493f0d561d14576f3705ff422c32fe305b2e6a', 'indexed': ['OraclePrice(str,str,Address,int)', 'USDICX', 'BandChain', 'cx7171e2f5653c1b9c000e24228276b8d24e84f10d'], 'data': ['0x84c5d90848544bb']} 

{'scoreAddress': 'cx764463174b4f7d44988968f93c08081881fcfc85', 'indexed': ['Snapshot(int)', '0x49'], 'data': []} 

{'scoreAddress': 'cxdf5d7067e425b3e1a2c96659b3fd91bd1b5dd0d9', 'indexed': ['Mint(Address,int,bytes)', 'cxbbf173efedf4338d0562edf67f3f563cd6f09b18'], 'data': ['0x152d02c7e14af6800000', '0x4e6f6e65']} 

{'scoreAddress': 'cxdf5d7067e425b3e1a2c96659b3fd91bd1b5dd0d9', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hx00000000000000

In [23]:
call_tx('loans', 'getAccountPositions', {'_owner': user1.get_address()})
call_tx('loans', 'getAccountPositions', {'_owner': user2.get_address()})


------------------------------------------------------------------------------------------------------------------
Reading getAccountPositions, with parameters {'_owner': 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db'} on the loans contract.
------------------------------------------------------------------------------------------------------------------
{'pos_id': '0x1', 'created': '0x5bf86488e38e9', 'address': 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db', 'snap_id': '0x47', 'snaps_length': '0x4', 'last_snap': '0x47', 'first day': '0x2', 'assets': {'sICX': '0xa22582e1ab15349167', 'bnUSD': '0x2b35a3416a687b8000'}, 'total_debt': '0x19d65e9ebdd00a9289', 'collateral': '0xa22582e1ab15349167', 'ratio': '0x57179c48eb1a3fe8', 'standing': 'Mining'}
------------------------------------------------------------------------------------------------------------------
Reading getAccountPositions, with parameters {'_owner': 'hx7a1824129a8fe803e45a3aae1c0e060399546187'} on the loans contract.
-----------

{'message': 'That address has no outstanding loans or deposited collateral.'}

In [29]:
# Gets the total number of borrowers in the system and iterates over them by index to update each one.

params = {}
call1 = CallBuilder().from_(wallet.get_address())\
                     .to(contracts['loans']['SCORE'])\
                     .method("borrowerCount")\
                     .params(params)\
                     .build()
total_borrowers = int(icon_service.call(call1), 0)
for i in range(total_borrowers):
    params = {'_index': i + 1}
    call = CallBuilder().from_(wallet.get_address())\
                        .to(contracts['loans']['SCORE'])\
                        .method("getPositionAddress")\
                        .params(params)\
                        .build()
    params = {'_owner': icon_service.call(call)}
    transaction = CallTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(contracts['loans']['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method("updateStanding")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    sleep(1)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Status: 1
{'scoreAddress': 'cxdf5d7067e425b3e1a2c96659b3fd91bd1b5dd0d9', 'indexed': ['Mint(Address,int,bytes)', 'cxbbf173efedf4338d0562edf67f3f563cd6f09b18'], 'data': ['0x152d02c7e14af6800000', '0x4e6f6e65']} 

{'scoreAddress': 'cxdf5d7067e425b3e1a2c96659b3fd91bd1b5dd0d9', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hx0000000000000000000000000000000000000000', 'cxbbf173efedf4338d0562edf67f3f563cd6f09b18', '0x152d02c7e14af6800000'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cxdf5d7067e425b3e1a2c96659b3fd91bd1b5dd0d9', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cxbbf173efedf4338d0562edf67f3f563cd6f09b18', 'cxf2c45e05c329bd9c0dd7780b8f14a15a1eaa78ff', '0x43c33c1937564800000'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cxdf5d7067e425b3e1a2c96659b3fd91bd1b5dd0d9', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cxbbf173efedf4338d0562edf67f3f563cd6f09b18', 'cxedf87c1cd6811891016606c403418cf967340a51', '0x10f0cf064dd59200000'], 'data': ['0x4e6f6e65']} 

{'scoreAddress

In [20]:
#2. redeem 50 ICD from user1 that  have 50 bnUSD  on Balanced. 
# a.deposite 800icx and mint 50 bnusd to both user1 and user2
# b.user2 tries to retire 50 bnusd(actual value of bnusd of user2 50.5 bnusd) 
send_tx('loans', 800*ICX, 'addCollateral', {'_asset': 'bnUSD', '_amount': 50 * ICX}, user1)
send_tx('loans', 800*ICX, 'addCollateral', {'_asset': 'bnUSD', '_amount': 50 * ICX}, user2)

call_tx('loans', 'getAccountPositions', {'_owner': user1.get_address()})
call_tx('loans', 'getAccountPositions', {'_owner': user2.get_address()})


------------------------------------------------------------------------------------------------------------------
Calling addCollateral, with parameters {'_asset': 'bnUSD', '_amount': 50000000000000000000} on the loans contract.
------------------------------------------------------------------------------------------------------------------


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cx63cfe2f8703dbedfc4dd80d93aa7ea917d498c06', 'indexed': ['ICXTransfer(Address,Address,int)', 'cx63cfe2f8703dbedfc4dd80d93aa7ea917d498c06', 'cx7dae0b285450d0abbfbc430c37fb480f1036a73d', '0x2b5e3af16b18800000'], 'data': []} 

{'scoreAddress': 'cxb75a7131820f8663ea14c1769374ac3e76bf5110', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cxb75a7131820f8663ea14c1769374ac3e76bf5110', 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db', '0x2b5e3af16b18800000'], 'data': ['0x7b226d6574686f64223a225f6465706f7369745f616e645f626f72726f77222c22706172616d73223a7b225f73656e646572223a22687865363264313166613139613065383537356164393266303662633866643432656462666532376462222c225f6173736574223a22626e555344222c225f616d6f756e74223a35303030303030303030303030303030303030307d7d']} 

{'scoreAddress': 'cxb75a7131820f8663ea14c1769374ac3e76bf5110', 'indexed': ['Mint

{'pos_id': '0x1', 'created': '0x5bdca345b8e0c', 'address': 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db', 'snap_id': '0x1', 'snaps_length': '0x1', 'last_snap': '0x1', 'first day': '0x1', 'assets': {'sICX': '0x2b5e3af16b18800000', 'bnUSD': '0x2bcd40a70853a0000'}, 'replay_index': '0x0', 'events_behind': '0x0', 'total_debt': '0x1a3107502244a0ee3', 'ratio': '0x16fa9b49c5a16dc0c', 'standing': 'Mining'}
------------------------------------------------------------------------------------------------------------------
Reading getAccountPositions, with parameters {'_owner': 'hx7a1824129a8fe803e45a3aae1c0e060399546187'} on the loans contract.
------------------------------------------------------------------------------------------------------------------
{'pos_id': '0x2', 'created': '0x5bdca347a1807', 'address': 'hx7a1824129a8fe803e45a3aae1c0e060399546187', 'snap_id': '0x1', 'snaps_length': '0x1', 'last_snap': '0x1', 'first day': '0x1', 'assets': {'sICX': '0x2b5e3af16b18800000', 'bnUSD': '0x2bcd

In [25]:
#2. redeem 50.5 bnusd from user1 that  have 50 bnUSD  on Balanced. 


params = {'_to': user2.get_address(), '_value': 50500000000000000000}
transaction = CallTransactionBuilder()\
    .from_(user1.get_address())\
    .to(contracts['bnUSD']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, user1)
tx_hash = icon_service.send_transaction(signed_transaction)
sleep(2)
params = {"method": "_retire_asset", "params": {}}
data = json.dumps(params).encode("utf-8")
params = {'_to': contracts['loans']['SCORE'], '_value': 50500000000000000000, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(user2.get_address())\
    .to(contracts['bnUSD']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, user2)
tx_hash = icon_service.send_transaction(signed_transaction)
sleep(1)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Status: 1
{'scoreAddress': 'cxbfa6fbf061d1d6bc9b55034efd2db3405ee7e0d1', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cx63cfe2f8703dbedfc4dd80d93aa7ea917d498c06', 'cxbfa6fbf061d1d6bc9b55034efd2db3405ee7e0d1', '0x6f05b59d3b20000'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cxbfa6fbf061d1d6bc9b55034efd2db3405ee7e0d1', 'indexed': ['Burn(Address,int)', 'cx63cfe2f8703dbedfc4dd80d93aa7ea917d498c06'], 'data': ['0x6f05b59d3b20000']} 

{'scoreAddress': 'cx63cfe2f8703dbedfc4dd80d93aa7ea917d498c06', 'indexed': ['LoanRepaid(Address,str,int,str)', 'hx7a1824129a8fe803e45a3aae1c0e060399546187', 'bnUSD', '0x6f05b59d3b20000'], 'data': ['Loan of 0.5 bnUSD repaid to Balanced.']} 

{'scoreAddress': 'cxbfa6fbf061d1d6bc9b55034efd2db3405ee7e0d1', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hx7a1824129a8fe803e45a3aae1c0e060399546187', 'cx63cfe2f8703dbedfc4dd80d93aa7ea917d498c06', '0x6f05b59d3b20000'], 'data': ['0x7b226d6574686f64223a20225f7265746972655f6173736574222c2022706172616d73223a207b7d

In [26]:
# check the account position after retiring
call_tx('loans', 'getAccountPositions', {'_owner': user1.get_address()})
call_tx('loans', 'getAccountPositions', {'_owner': user2.get_address()})

------------------------------------------------------------------------------------------------------------------
Reading getAccountPositions, with parameters {'_owner': 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db'} on the loans contract.
------------------------------------------------------------------------------------------------------------------
{'pos_id': '0x1', 'created': '0x5bdca345b8e0c', 'address': 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db', 'snap_id': '0x1', 'snaps_length': '0x1', 'last_snap': '0x1', 'first day': '0x1', 'assets': {'sICX': '0x2b5e3af16b18800000', 'bnUSD': '0x2bcd40a70853a0000'}, 'replay_index': '0x0', 'events_behind': '0x0', 'total_debt': '0x1a3107502244a0ee3', 'ratio': '0x16fa9b49c5a16dc0c', 'standing': 'Mining'}
------------------------------------------------------------------------------------------------------------------
Reading getAccountPositions, with parameters {'_owner': 'hx7a1824129a8fe803e45a3aae1c0e060399546187'} on the loans contract.
-----

In [31]:
# 3.no bad debt for asset, retiring 50 bnUSD from account with a debt position > 50.
# a. user1 has  800 ICX and 50 bnusd loan 
# b.user2 has 800icx and 70 bnusd loan

call_tx('loans', 'getAccountPositions', {'_owner': user2.get_address()})
send_tx('loans', 0, 'originateLoan', {'_asset':'bnUSD', '_amount':50 *ICX}, user2)
call_tx('loans', 'getAccountPositions', {'_owner': user1.get_address()})
call_tx('loans', 'getAccountPositions', {'_owner': user2.get_address()})

------------------------------------------------------------------------------------------------------------------
Reading getAccountPositions, with parameters {'_owner': 'hx7a1824129a8fe803e45a3aae1c0e060399546187'} on the loans contract.
------------------------------------------------------------------------------------------------------------------
{'pos_id': '0x2', 'created': '0x5bdca347a1807', 'address': 'hx7a1824129a8fe803e45a3aae1c0e060399546187', 'snap_id': '0x2', 'snaps_length': '0x2', 'last_snap': '0x2', 'first day': '0x1', 'assets': {'sICX': '0x2b5e3af16b18800000', 'bnUSD': '0x11f452c53a2960000', 'BALN': '0x0'}, 'replay_index': '0x0', 'events_behind': '0x0', 'total_debt': '0xabc65d95ea6041eb', 'ratio': '0x380f495fe0e7d064a', 'standing': 'Mining'}
------------------------------------------------------------------------------------------------------------------
Calling originateLoan, with parameters {'_asset': 'bnUSD', '_amount': 50000000000000000000} on the loans contract.
-

In [36]:
# c.user2 tries to retire 50 bnusd

test_cases =  { 
    
    "stories": [ {
      "description": "User2 tres to retire 50 bnusd",
      "actions": {
          "sender":"user2",
          "first_meth": "transfer",
          "second_meth":"transfer",
          "deposited_icx":"0",
          "first_params":{'_to': user2.get_address(), '_value': 50 * ICX},
          "second_params":{"method": "_retire_asset", "params": {},'_to': contracts['loans']['SCORE'], '_value': 50 * ICX},
          "expected_bnusd_wallet":"20700000000000000000"
          
      }
    }     

  ]
}

for case in test_cases['stories']:
    print(case['description'])
    if case['actions']['sender'] == 'user1':
        wallet_address = user1.get_address()
        wallet = user1
    else:
        wallet_address = user2.get_address()
        wallet = user2
    
    _to = contracts['bnUSD']['SCORE']
    meth1 = case['actions']['first_meth']
    meth2 = case['actions']['second_meth']
    val = int(case['actions']['deposited_icx'])
    data1 = case['actions']['first_params']
    first_params = {"_to": data1['_to'], "_value": data1['_value']}
    
    data2 = case['actions']['second_params']
    params = {"method": data2['method'], "params": data2['params']}
    data = json.dumps(params).encode("utf-8")
    second_params = {'_to': data2['_to'], '_value': data2['_value'], '_data': data}


    transaction = CallTransactionBuilder()\
        .from_(user1.get_address())\
        .to(_to)\
        .value(val)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(meth1)\
        .params(first_params)\
        .build()
    signed_transaction = SignedTransaction(transaction, user1)
    tx_hash = icon_service.send_transaction(signed_transaction)
    sleep(2)

    transaction = CallTransactionBuilder()\
        .from_(user2.get_address())\
        .to(_to)\
        .value(val)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(meth2)\
        .params(second_params)\
        .build()
    signed_transaction = SignedTransaction(transaction, user2)
    tx_hash = icon_service.send_transaction(signed_transaction)
    sleep(1)

    res = get_tx_result(tx_hash)
    print(f'Status: {res["status"]}')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        print(f'Failure: {res["failure"]}')

User1 delegates 100% of it's votes to hx4917005bf4b8188b9591da520fc1c6dab8fe717f
Status: 1
{'scoreAddress': 'cx1940cd3c1c873a0a5fcbec084acbde151639194e', 'indexed': ['Mint(Address,int,bytes)', 'cx1940cd3c1c873a0a5fcbec084acbde151639194e'], 'data': ['0x152d02c7e14af6800000', '0x4e6f6e65']} 

{'scoreAddress': 'cx1940cd3c1c873a0a5fcbec084acbde151639194e', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cx1940cd3c1c873a0a5fcbec084acbde151639194e', 'cxbebe4baa27910566e5340b23d315db8bb84f24ae', '0x152d02c7e14af6800000'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cx1940cd3c1c873a0a5fcbec084acbde151639194e', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cxbebe4baa27910566e5340b23d315db8bb84f24ae', 'cxe1bf6cfbcad5c0c067590d4dfe8fc74dc62c66e7', '0x43c33c1937564800000'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cx1940cd3c1c873a0a5fcbec084acbde151639194e', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cxbebe4baa27910566e5340b23d315db8bb84f24ae', 'cxc23e00aa79121c3145287e1abc4d

In [46]:
call_tx('loans', 'getAccountPositions', {'_owner': user1.get_address()})
res = call_tx('loans', 'getAccountPositions', {'_owner': user2.get_address()})
print(res['assets']['bnUSD'])
if (res['assets']['bnUSD']) == hex(21200000000000000000):
    print('success')
else:
    print('fail')

------------------------------------------------------------------------------------------------------------------
Reading getAccountPositions, with parameters {'_owner': 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db'} on the loans contract.
------------------------------------------------------------------------------------------------------------------
{'pos_id': '0x1', 'created': '0x5bdca345b8e0c', 'address': 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db', 'snap_id': '0x1', 'snaps_length': '0x1', 'last_snap': '0x1', 'first day': '0x1', 'assets': {'sICX': '0x2b5e3af16b18800000', 'bnUSD': '0x2bcd40a70853a0000'}, 'replay_index': '0x0', 'events_behind': '0x0', 'total_debt': '0x0', 'ratio': '0x0', 'standing': 'Indeterminate'}
------------------------------------------------------------------------------------------------------------------
Reading getAccountPositions, with parameters {'_owner': 'hx7a1824129a8fe803e45a3aae1c0e060399546187'} on the loans contract.
------------------------------

In [22]:
# 4.Retiring 50 bnUSD, with bad_debt > 50 bnUSD for the asset, retired from account without a position on Balanced, liquidation pool > 1.1x the value of the retired asset.
# a. Liquidate an account with a bad_debt more than 50 bnusd

# Try liquidating an account user1

# 1. Deposit collateral to user1 wallet
# 2. Check the account standing after depositing collateral - Should be 'No Debt', and
#    holding '0x2a6f1a22364bbe8000' worth of sICX.
# 3a. Mints 200 ICD to the test address without checking collateralization ratio
#     With the above collateral deposit it will put the position in a standing of Liquidation.
# 4. Check the new standing of the account. Should have added '0xad78ebc5ac6200000' ICD and have standing of 'Liquidate'.

call_tx('loans', 'getAccountPositions', {'_owner': user1.get_address()})
send_tx('loans', 500 * ICX, 'addCollateral', {'_asset': '', '_amount': 0}, user1)
call_tx('loans', 'getAccountPositions', {'_owner': user1.get_address()})
send_tx('loans', 0, 'add_bad_test_position', {}, user1)
call_tx('loans', 'getAccountPositions', {'_owner': user1.get_address()})
send_tx('loans', 0, 'add_bad_test_position', {}, user1)
send_tx('loans', 0, 'add_bad_test_position', {}, user1)
call_tx('loans', 'getAccountPositions', {'_owner': user1.get_address()})


------------------------------------------------------------------------------------------------------------------
Reading getAccountPositions, with parameters {'_owner': 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db'} on the loans contract.
------------------------------------------------------------------------------------------------------------------
{'pos_id': '0x1', 'created': '0x5bdde2116b206', 'address': 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db', 'snap_id': '0x1', 'snaps_length': '0x1', 'last_snap': '0x1', 'first day': '0x1', 'assets': {'sICX': '0x1b1ae4d6e2ef500000', 'bnUSD': '0x15af1d78b58c400000'}, 'replay_index': '0x0', 'events_behind': '0x0', 'total_debt': '0xcf75231cf103b6430', 'ratio': '0x1d02ca7389823429', 'standing': 'Locked'}
------------------------------------------------------------------------------------------------------------------
Calling add_bad_test_position, with parameters {} on the loans contract.
---------------------------------------------------------

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cx30f2b3443542fc7aec4956c922d95f116da521ae', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cx30f2b3443542fc7aec4956c922d95f116da521ae', 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db', '0xad78ebc5ac6200000'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cx30f2b3443542fc7aec4956c922d95f116da521ae', 'indexed': ['Mint(Address,int,bytes)', 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db'], 'data': ['0xad78ebc5ac6200000', '0x4e6f6e65']} 

------------------------------------------------------------------------------------------------------------------
Reading getAccountPositions, with parameters {'_owner': 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db'} on the loans contract.
------------------------------------------------------------------------------------------------------------------
{'pos_id': '0x1', 'created': '0x5bdde2116b206', 'addre

In [23]:
# 5. Liquidate the account position.
# 6. Check the standing of the account after liquidation. Should now have zero balance for sICX and ICD.
# 7. Checking the debts should show the sum of the borrower debt and the bad debt equals the total supply.

call_tx('loans', 'getAccountPositions', {'_owner': user1.get_address()})
send_tx('loans', 0, 'liquidate', {'_owner': user1.get_address()}, user1)
call_tx('loans', 'getAccountPositions', {'_owner': user1.get_address()})
call_tx('loans', 'checkDebts', {})


------------------------------------------------------------------------------------------------------------------
Reading getAccountPositions, with parameters {'_owner': 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db'} on the loans contract.
------------------------------------------------------------------------------------------------------------------
{'pos_id': '0x1', 'created': '0x5bdde2116b206', 'address': 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db', 'snap_id': '0x1', 'snaps_length': '0x1', 'last_snap': '0x1', 'first day': '0x1', 'assets': {'sICX': '0x1b1ae4d6e2ef500000', 'bnUSD': '0x2086ac351052600000'}, 'replay_index': '0x0', 'events_behind': '0x0', 'total_debt': '0x1372fb4ab698591648', 'ratio': '0x135731a25bac22c6', 'standing': 'Liquidate'}
------------------------------------------------------------------------------------------------------------------
Calling liquidate, with parameters {'_owner': 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db'} on the loans contract.
-----------

In [25]:
# 8. after liquidating we have a bad_debt of 600 bnusd and supply of 600 bnusd.Now, we retire 50 bnusd from user2 
#   that doesn't have position on balanced.



params = {'_to': user2.get_address(), '_value': 50 * ICX}
transaction = CallTransactionBuilder()\
    .from_(user1.get_address())\
    .to(contracts['bnUSD']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, user1)
tx_hash = icon_service.send_transaction(signed_transaction)
sleep(2)
params = {"method": "_retire_asset", "params": {}}
data = json.dumps(params).encode("utf-8")
params = {'_to': contracts['loans']['SCORE'], '_value': 50 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(user2.get_address())\
    .to(contracts['bnUSD']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, user2)
tx_hash = icon_service.send_transaction(signed_transaction)
sleep(1)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Status: 1
{'scoreAddress': 'cx30f2b3443542fc7aec4956c922d95f116da521ae', 'indexed': ['OraclePrice(str,str,Address,int)', 'USDICX', 'BandChain', 'cx7171e2f5653c1b9c000e24228276b8d24e84f10d'], 'data': ['0x84c5d90848544bb']} 

{'scoreAddress': 'cx30f2b3443542fc7aec4956c922d95f116da521ae', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cxe49964351f78130791764def7b86a395e561e750', 'cx30f2b3443542fc7aec4956c922d95f116da521ae', '0x2b26b8169c7af0000'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cx30f2b3443542fc7aec4956c922d95f116da521ae', 'indexed': ['Burn(Address,int)', 'cxe49964351f78130791764def7b86a395e561e750'], 'data': ['0x2b26b8169c7af0000']} 

{'scoreAddress': 'cx3d607dbc36d63c190a6fa130392794c7e1b1bf3a', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cxe49964351f78130791764def7b86a395e561e750', 'hx7a1824129a8fe803e45a3aae1c0e060399546187', '0x1c61fe6b7edc9eadf'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cxe49964351f78130791764def7b86a395e561e750', 'indexed': ['TokenTrans

In [26]:
call_tx('loans', 'getAccountPositions', {'_owner': user1.get_address()})
call_tx('loans', 'getAccountPositions', {'_owner': user2.get_address()})
call_tx('loans', 'checkDebts', {})

------------------------------------------------------------------------------------------------------------------
Reading getAccountPositions, with parameters {'_owner': 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db'} on the loans contract.
------------------------------------------------------------------------------------------------------------------
{'pos_id': '0x1', 'created': '0x5bdde2116b206', 'address': 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db', 'snap_id': '0x1', 'snaps_length': '0x1', 'last_snap': '0x1', 'first day': '0x1', 'assets': {'sICX': '0x0', 'bnUSD': '0x0'}, 'replay_index': '0x0', 'events_behind': '0x0', 'total_debt': '0x0', 'ratio': '0x0', 'standing': 'Zero'}
------------------------------------------------------------------------------------------------------------------
Reading getAccountPositions, with parameters {'_owner': 'hx7a1824129a8fe803e45a3aae1c0e060399546187'} on the loans contract.
------------------------------------------------------------------------

Testing dead market

In [27]:
# 1.Try originating loan from the market which is dead, the transaction fails.
call_tx('loans', 'getAccountPositions', {'_owner': user1.get_address()})
send_tx('loans', 0, 'originateLoan', {'_asset':'bnUSD', '_amount':50 *ICX}, user1)
call_tx('loans', 'getAccountPositions', {'_owner': user1.get_address()})


------------------------------------------------------------------------------------------------------------------
Reading getAccountPositions, with parameters {'_owner': 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db'} on the loans contract.
------------------------------------------------------------------------------------------------------------------
{'pos_id': '0x1', 'created': '0x5bdde2116b206', 'address': 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db', 'snap_id': '0x1', 'snaps_length': '0x1', 'last_snap': '0x1', 'first day': '0x1', 'assets': {'sICX': '0x0', 'bnUSD': '0x0'}, 'replay_index': '0x0', 'events_behind': '0x0', 'total_debt': '0x0', 'ratio': '0x0', 'standing': 'Zero'}
------------------------------------------------------------------------------------------------------------------
Calling originateLoan, with parameters {'_asset': 'bnUSD', '_amount': 50000000000000000000} on the loans contract.
----------------------------------------------------------------------------------

In [39]:
#  Now we try to revive the dead market
# 1. Test with retiring less than enough to revive the market with bad_debt 550.25 bnusd 
# i.e user2 retires 100 bnusd

test_cases =  { 
    
    "stories": [ {
      "description": "User2 tres to retire 100 bnusd",
      "actions": {
          "sender":"user2",
          "first_meth": "transfer",
          "second_meth":"transfer",
          "deposited_icx":"0",
          "first_params":{'_to': user2.get_address(), '_value': 100 * ICX},
          "second_params":{"method": "_retire_asset", "params": {},'_to': contracts['loans']['SCORE'], '_value': 100 * ICX},
          "expected_bnusd_wallet":"20700000000000000000"
          
      }
    },
        {
            "description": "User2 tries to retire 100 bnusd",
              "actions": {
                  "sender":"user2",
                  "first_meth": "transfer",
                  "second_meth":"transfer",
                  "deposited_icx":"0",
                  "first_params":{'_to': user2.get_address(), '_value': 251750000000000000000},
                  "second_params":{"method": "_retire_asset", "params": {},'_to': contracts['loans']['SCORE'], '_value': 251250000000000000000},
                  "expected_bnusd_wallet":"20700000000000000000"

      }
        }

  ]
}

for case in test_cases['stories']:
    print(case['description'])
    _to = contracts['bnUSD']['SCORE']
    meth1 = case['actions']['first_meth']
    meth2 = case['actions']['second_meth']
    val = int(case['actions']['deposited_icx'])
    data1 = case['actions']['first_params']
    first_params = {"_to": data1['_to'], "_value": data1['_value']}
    
    data2 = case['actions']['second_params']
    params = {"method": data2['method'], "params": data2['params']}
    data = json.dumps(params).encode("utf-8")
    second_params = {'_to': data2['_to'], '_value': data2['_value'], '_data': data}


    transaction = CallTransactionBuilder()\
        .from_(user1.get_address())\
        .to(_to)\
        .value(val)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(meth1)\
        .params(first_params)\
        .build()
    signed_transaction = SignedTransaction(transaction, user1)
    tx_hash = icon_service.send_transaction(signed_transaction)
    sleep(2)

    transaction = CallTransactionBuilder()\
        .from_(user2.get_address())\
        .to(_to)\
        .value(val)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(meth2)\
        .params(second_params)\
        .build()
    signed_transaction = SignedTransaction(transaction, user2)
    tx_hash = icon_service.send_transaction(signed_transaction)
    sleep(1)

    res = get_tx_result(tx_hash)
    print(f'Status: {res["status"]}')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        print(f'Failure: {res["failure"]}')
    
    tes= call_tx('loans', 'checkDebts', {})
    if tes['system']['bad_debt'] != hex(0):
        print("Market still dead")
    else:
        print("Market revived")
    

User2 tres to retire 100 bnusd
Status: 0
Failure: {'code': 1, 'message': 'Insufficient balance.'}
------------------------------------------------------------------------------------------------------------------
Reading checkDebts, with parameters {} on the loans contract.
------------------------------------------------------------------------------------------------------------------
{'system': {'debts': '0x0', 'bad_debt': '0x2d1b26f18ea386000', 'supply': '0x2d1b26f18ea386000', 'diff': '0x0', 'remaining': '0x0'}, 'events': {}}
Market still dead


In [40]:
call_tx('loans', 'getAccountPositions', {'_owner': user1.get_address()})
send_tx('loans', 800 * ICX, 'addCollateral', {'_asset': '', '_amount': 0}, user1)
send_tx('loans', 0, 'originateLoan', {'_asset':'bnUSD', '_amount':50 *ICX}, user1)
call_tx('loans', 'getAccountPositions', {'_owner': user1.get_address()})

call_tx('loans', 'checkDebts', {})


------------------------------------------------------------------------------------------------------------------
Reading getAccountPositions, with parameters {'_owner': 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db'} on the loans contract.
------------------------------------------------------------------------------------------------------------------
{'pos_id': '0x1', 'created': '0x5bdde2116b206', 'address': 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db', 'snap_id': '0x1', 'snaps_length': '0x1', 'last_snap': '0x1', 'first day': '0x1', 'assets': {'sICX': '0x0', 'bnUSD': '0x0'}, 'replay_index': '0x0', 'events_behind': '0x0', 'total_debt': '0x0', 'ratio': '0x0', 'standing': 'Indeterminate'}
------------------------------------------------------------------------------------------------------------------
Calling addCollateral, with parameters {'_asset': '', '_amount': 0} on the loans contract.
-------------------------------------------------------------------------------------------------

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cxe49964351f78130791764def7b86a395e561e750', 'indexed': ['ICXTransfer(Address,Address,int)', 'cxe49964351f78130791764def7b86a395e561e750', 'cxcc3158173154e905c0ab933ac3b35bdd3694dce9', '0x2b5e3af16b18800000'], 'data': []} 

{'scoreAddress': 'cx3d607dbc36d63c190a6fa130392794c7e1b1bf3a', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cx3d607dbc36d63c190a6fa130392794c7e1b1bf3a', 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db', '0x2b5e3af16b18800000'], 'data': ['0x7b226d6574686f64223a225f6465706f7369745f616e645f626f72726f77222c22706172616d73223a7b225f73656e646572223a22687865363264313166613139613065383537356164393266303662633866643432656462666532376462222c225f6173736574223a22222c225f616d6f756e74223a307d7d']} 

{'scoreAddress': 'cx3d607dbc36d63c190a6fa130392794c7e1b1bf3a', 'indexed': ['Mint(Address,int,bytes)', 'hxe62d11fa19a0e8575ad92f0

{'system': {'debts': '0x2bcd40a70853a0000',
  'bad_debt': '0x2d1b26f18ea386000',
  'supply': '0x58e8679896f726000',
  'diff': '0x0',
  'remaining': '0x0'},
 'events': {}}